Casava 1.8 FASTQ ID format from 
https://en.wikipedia.org/wiki/FASTQ_format

@EAS139:136:FC706VJ:2:2104:15343:197393 1:Y:18:ATCACG
                                    
* EAS139	the unique instrument name
* 136	the run id
* FC706VJ	the flowcell id
* 2	flowcell lane
* 2104	tile number within the flowcell lane
* 15343	'x'-coordinate of the cluster within the tile
* 197393	'y'-coordinate of the cluster within the tile
* 1	the member of a pair, 1 or 2 (paired-end or mate-pair reads only)
* Y	Y if the read is filtered, N otherwise
* 18	0 when none of the control bits are on, otherwise it is an even number
* ATCACG	index sequence

The sequence of quality values (from left to right):
```
!"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~
```


In [11]:
import random

nucleotides = ["A", "C", "T", "G"]

def random_nucleotide_sequence(length):
    return "".join(random.choices(nucleotides, k=length))

print(random_nucleotide_sequence(10))
print(random_nucleotide_sequence(1))
print(random_nucleotide_sequence(100))

                

AGTAGTCGAT
G
CTGCGCGACCGTATCCTTTCGCGGGGCTCATGGGGCAAGGTACGAACAGTAGCATGAAACACCGGCACAGCTATAGACTGCTAGATCCACTCGATATCAT


In [39]:
quality_chars = "!\"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~"

def random_qualities(
        length,
        min_quality=0,
        max_quality=len(quality_chars)):
    return "".join(random.choices(quality_chars[min_quality:max_quality], k=length))

print(random_qualities(10))
print(random_qualities(1))
print(random_qualities(100))


print(random_qualities(10, min_quality=20))
print(random_qualities(1, min_quality=20))

print(random_qualities(100, min_quality=20))

RbnCspL80E
z
Ax0cz#og>4dX@,`Y<9(`_Ag5fVg~M)!?J%:17,]Vk/,90Bf7yt%>D9YLq[c;!*NvI&@CoERkvZ|#+a&BF#@N=UATAJtyU@O(//|T
RR?|aUt?~6
x
pa6C=5?MLnp=mnW9r9OLa=DbT<bjszavuGzvsULL?:[DxkSqWNF[Kar;8aLfNuTNJwvgMvhwDcJ:HC>TDmM>=?5LCN|_>W6`B5~A


In [201]:
from Bio.Seq import reverse_complement

class ReadGenerator(object):
    def __init__(
            self,
            instrument_name="HISEQ1",
            run_id=1,
            flowcell_id="FC706VJ",
            lane=1,
            max_tile_number=9999,
            max_x_coord=2 * 10 ** 4,
            max_y_coord=2 * 10 ** 5,
            pair_number=1,
            filtered_reads=False,
            control_bits=18,
            index_length=6,
            min_base_quality=20,
            max_base_quality=40):
        self.instrument_name = instrument_name
        self.run_id = run_id
        self.flowcell_id = flowcell_id
        self.lane = lane
        self.max_tile_number = max_tile_number
        self.max_x_coord = max_x_coord
        self.max_y_coord = max_y_coord
        self.pair_number = pair_number
        self.filtered_reads = filtered_reads
        self.control_bits = control_bits
        self.index_length = index_length
        self.min_base_quality = min_base_quality
        self.max_base_quality = max_base_quality
        
    def _random_index(self):
        return random_nucleotide_sequence(self.index_length)
    
    def generate_reads(self, sequence, count, read_length=50):
        """
        Returns list of tuples whose elements are
            1) id string
            2) read sequence
            3 ) quality scores
        """
        assert len(sequence) >= read_length
        fastq_entries = []
        index = self._random_index()
        for i in range(count):
            offset = random.randint(0, len(sequence) - read_length)
            read_seq = sequence[offset:offset + read_length]
            qualities = random_qualities(
                length=read_length, 
                min_quality=self.min_base_quality,
                max_quality=self.max_base_quality)
            tile_number = random.randint(1, self.max_tile_number)
            x_coord = random.randint(1, self.max_x_coord)
            y_coord = random.randint(1, self.max_y_coord)
            id_string = "@%s:%d:%s:%d:%d:%d:%d %d:%s:%d:%s length=%d" % (
                self.instrument_name,
                self.run_id,
                self.flowcell_id,
                self.lane, 
                tile_number,
                x_coord,
                y_coord,
                self.pair_number,
                "Y" if self.filtered_reads else "N",
                self.control_bits,
                index,
                read_length)
            fastq_entry = (id_string, read_seq, qualities)
            fastq_entries.append(fastq_entry)
        return fastq_entries
    
    def generate_fastq_string(self, sequence, count, read_length=50):
        fastq_entries = self.generate_reads(sequence=sequence, count=count, read_length=read_length)
        return "\n".join([
            "%s\n%s\n+\n%s" % (id_string, seq, qual) for (id_string, seq, qual) in fastq_entries])
    
    def generate_fastq_strings_for_variant(self, variant, ref_count, alt_count, read_length=50):
        effect = variant.effects().top_priority_effect()
        transcript = effect.transcript
        transcript_sequence = transcript.sequence
        variant_start_offset = transcript.spliced_offset(variant.start)
        variant_end_offset = transcript.spliced_offset(variant.end)
        
        prefix_start_offset = max(0, variant_start_offset - read_length)
        prefix = transcript_sequence[prefix_start_offset:variant_start_offset]
        suffix = transcript_sequence[
            variant_end_offset + 1:variant_end_offset + read_length]
        
        
        if transcript.on_negative_strand:
            ref = reverse_complement(variant.ref)
            alt = reverse_complement(variant.alt)
        else:
            ref = variant.ref
            alt = variant.alt
        
        ref_sequence = prefix + ref + suffix
        alt_sequence = prefix + alt + suffix
        
        expected_ref_sequence = transcript_sequence[
            prefix_start_offset:
            variant_end_offset + read_length]
        
        if ref_sequence != expected_ref_sequence:
            raise ValueError(
                "Ref sequence (length %d, type=%s) not same as prefix+ref+suffix (length %d, type=%s)" % (
                len(expected_ref_sequence), 
                type(expected_ref_sequence),
                len(ref_sequence),
                type(ref_sequence)))
        
        ref_fastq_string = self.generate_fastq_string(
            sequence=ref_sequence,
            count=ref_count,
            read_length=read_length)
        alt_fastq_string = self.generate_fastq_string(
            sequence=alt_sequence,
            count=alt_count,
            read_length=read_length)
        return ref_fastq_string, alt_fastq_string


In [202]:
r = ReadGenerator()

In [203]:
print(r.generate_fastq_string("ACTGAACCTTGGAAACCCTTTGGG", count=2, read_length=5))

@HISEQ1:1:FC706VJ:1:270:17771:93156 1:N:18:AGTAAC length=5
GAAAC
+
@C5GG
@HISEQ1:1:FC706VJ:1:4267:16539:72309 1:N:18:AGTAAC length=5
AAACC
+
6:;;@


In [204]:
from varcode import Variant

In [205]:
idh1_r132h = Variant(2, 209113112, "C", "T", ensembl="grch37")

In [206]:
idh1_r132h.effects().top_priority_effect()

Substitution(variant=Variant(contig='2', start=209113112, ref='C', alt='T', reference_name='GRCh37'), transcript_name=IDH1-006, transcript_id=ENST00000415913, effect_description=p.R132H)

In [207]:
ref_fastq, alt_fastq = r.generate_fastq_strings_for_variant(variant=idh1_r132h, ref_count=50, alt_count=100, read_length=75)

In [208]:
print(ref_fastq)

@HISEQ1:1:FC706VJ:1:9656:9490:92913 1:N:18:GAGCTA length=75
CGTCATGCTTATGGGGATCAATACAGAGCAACTGATTTTGTTGTTCCTGGGCCTGGAAAAGTAGAGATAACCTAC
+
8DH@A8;@><FAFD7>8<:5675H6@6796FE;D;?HA>8GDD?95A6>5>DC5=C69;959::9ACD@78DAG>
@HISEQ1:1:FC706VJ:1:1013:13785:19553 1:N:18:GAGCTA length=75
CCCGGCTTGTGAGTGGATGGGTAAAACCTATCATCATAGGTCGTCATGCTTATGGGGATCAATACAGAGCAACTG
+
;96?7BA>5EE7HD<CG@G>BCCDHCFDF:FA=8>@889?5F68>GH57;7A6=<E?=:GHG8>@=:6=;=BE6@
@HISEQ1:1:FC706VJ:1:2151:18270:179681 1:N:18:GAGCTA length=75
ACCTATCATCATAGGTCGTCATGCTTATGGGGATCAATACAGAGCAACTGATTTTGTTGTTCCTGGGCCTGGAAA
+
>65<<ED6><?=G6??DD9>@A8=BB8CGE??9==7>C=ACE?E;5:5<@FF=E?;6F?@CDF:6;E9888FG@9
@HISEQ1:1:FC706VJ:1:2250:9620:142493 1:N:18:GAGCTA length=75
AGCCATTATCTGCAAAAATATCCCCCGGCTTGTGAGTGGATGGGTAAAACCTATCATCATAGGTCGTCATGCTTA
+
;E6?<8C>H8<ABEH<>GB=@?F@=D<8@H@98EF@8;75F=5AF<C9==7D9D876B7?8DE<CFE<?=<HGC:
@HISEQ1:1:FC706VJ:1:1002:16761:146440 1:N:18:GAGCTA length=75
CCCCCGGCTTGTGAGTGGATGGGTAAAACCTATCATCATAGGTCGTCATGCTTATGGGGATCAATACAGAGCAAC
+


In [209]:
with open("id1h_r132h_normal.fastq", "w") as f:
    f.write(ref_fastq)
with open("id1h_r132h_tumor.fastq", "w") as f:
    f.write(alt_fastq)

In [210]:
!bwa mem chr2.fa id1h_r132h_normal.fastq > normal.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 50 sequences (3750 bp)...
[M::mem_process_seqs] Processed 50 reads in 0.012 CPU sec, 0.023 real sec
[main] Version: 0.7.15-r1140
[main] CMD: bwa mem chr2.fa id1h_r132h_normal.fastq
[main] Real time: 1.423 sec; CPU: 0.642 sec


In [211]:
!bwa mem chr2.fa id1h_r132h_tumor.fastq > tumor.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 100 sequences (7500 bp)...
[M::mem_process_seqs] Processed 100 reads in 0.011 CPU sec, 0.011 real sec
[main] Version: 0.7.15-r1140
[main] CMD: bwa mem chr2.fa id1h_r132h_tumor.fastq
[main] Real time: 0.405 sec; CPU: 0.380 sec


In [212]:
!sambamba view normal.sam -f bam -S  > normal.bam

In [ ]:
!sambamba view tumor.sam -f bam -S  > tumor.bam

In [116]:
!sambamba sort normal.bam > normal.sorted.bam

In [117]:
!sambamba sort tumor.bam > tumor.sorted.bam

In [118]:
!samtools mpileup -uf chr2.fa tumor.sorted.bam normal.sorted.bam | bcftools call -m  -v

[mpileup] 2 samples in 2 input files
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
<mpileup> Set max per-file depth to 4000
##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##samtoolsVersion=1.2+htslib-1.2
##samtoolsCommand=samtools mpileup -uf chr2.fa tumor.sorted.bam normal.sorted.bam
##reference=file://chr2.fa
##contig=<ID=chr2,length=243199373>
##ALT=<ID=X,Description="Represents allele(s) other than observed.">
##INFO=<ID=INDEL,Number=0,Type=Flag,Description="Indicates that the variant is an INDEL.">
##INFO=<ID=IDV,Number=1,Type=Integer,Description="Maximum number of reads supporting an indel">
##INFO=<ID=IMF,Number=1,Type=Float,Description="Maximum fraction of reads supporting an indel">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Raw read depth">
##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">
##IN